# Baseline (2022)

In [40]:
using CSV
using DataFrames
using JuMP
using Gurobi
using Statistics

# ---------------------------------------------
# 1. Load baseline EV registrations (non-scenario dataset)
#    cleaned_registration_ny_with_coords.csv
# ---------------------------------------------
reg_raw = CSV.read("../data/cleaned_reg_1128.csv", DataFrame)

# Make sure ZIP is an Int (often it shows up as Float64 or String)
if eltype(reg_raw.ZIP) <: AbstractFloat
    reg_raw.ZIP = round.(Int, reg_raw.ZIP)
elseif eltype(reg_raw.ZIP) <: AbstractString
    reg_raw.ZIP = parse.(Int, strip.(reg_raw.ZIP))
end

# If there are multiple rows per ZIP, aggregate vehicle_count
# (if it's already one row per ZIP, this just passes through)
reg_zip = combine(
    groupby(reg_raw, :ZIP),
    :N => sum => :vehicle_count,
)

println("Number of ZIPs in registration file: ", nrow(reg_zip))

# ---------------------------------------------
# 2. Stations (ID, Status Code, total_evse)
# ---------------------------------------------
stations = CSV.read("../data/cleaned_station.csv", DataFrame)

stations.total_evse =
    coalesce.(stations[!, "EV Level1 EVSE Num"], 0.0) .+
    coalesce.(stations[!, "EV Level2 EVSE Num"], 0.0) .+
    coalesce.(stations[!, "EV DC Fast Count"], 0.0)

stations_small = select(stations, :ID, :"Status Code", :total_evse)

# ---------------------------------------------
# 3. Distance matrix dij (ZIP × station)
#    Structure: demand_zip | 35272 | 35562 | ...
# ---------------------------------------------
dist_raw = CSV.read("../data/dij.csv", DataFrame)

# Map ZIP → row index in dist_raw
zip_to_row = Dict(dist_raw.demand_zip[r] => r for r in 1:nrow(dist_raw))

# All station columns in the distance matrix
station_cols = names(dist_raw)[2:end]

# Helper: column name -> Int station ID (works for Symbols/Strings)
station_id_from_colname(name) = parse(Int, String(name))

# Station IDs as they appear in dij.csv
station_ids_from_dist = [station_id_from_colname(c) for c in station_cols]

# Restrict to stations that appear in BOTH station table and distance matrix
ids_from_stations = Set(stations_small.ID)
candidate_station_ids = [sid for sid in station_ids_from_dist if sid in ids_from_stations]

println("Number of candidate stations (intersection): ",
        length(candidate_station_ids))

# Create J index set based on candidate stations
station_ids = candidate_station_ids
J = collect(1:length(station_ids))
nJ = length(J)

# Map station_id -> column index in dist_raw
col_idx_for_station = Dict{Int,Int}()
for (k, col_name) in enumerate(station_cols)
    sid = station_id_from_colname(col_name)
    if sid in station_ids
        col_idx_for_station[sid] = k + 1   # +1 because col 1 is :demand_zip
    end
end

# ---------------------------------------------
# 4. Align ZIPs between registration file and dij,
#    and build baseline demand vector D and distance matrix d
# ---------------------------------------------

# Keep only ZIPs that appear in BOTH reg_zip and dist_raw
valid_zips = sort(
    collect(
        intersect(Set(reg_zip.ZIP), Set(dist_raw.demand_zip))
    )
)

println("ZIPs in both registration & dij: ", length(valid_zips))

# Internal demand-node index set
zip_codes = valid_zips              # external labels for demand nodes
I = collect(1:length(zip_codes))    # internal indices 1..|I|
nI = length(I)

# Build a dictionary ZIP -> vehicle_count
veh_dict = Dict(row.ZIP => row.vehicle_count for row in eachrow(reg_zip))

# Parameter D[i]: expected daily charging demand at ZIP i
sessions_per_EV_per_day = 0.25
D = [sessions_per_EV_per_day * veh_dict[z] for z in zip_codes]

println("Total baseline EV sessions/day (sum D): ", sum(D))

# Build dense distance matrix d[i,j] (in km) aligned with I and J
d = Matrix{Float64}(undef, nI, nJ)

for (i_idx, z) in enumerate(zip_codes)
    if !haskey(zip_to_row, z)
        error("ZIP $z from registration file not found in dij.csv; ",
              "either fix dij.csv or drop this ZIP.")
    end
    row = zip_to_row[z]
    for (j_idx, sid) in enumerate(station_ids)
        col = col_idx_for_station[sid]
        d[i_idx, j_idx] = dist_raw[row, col]
    end
end

println("Built distance matrix d of size ", size(d))

Number of ZIPs in registration file: 1064
Number of candidate stations (intersection): 5594
ZIPs in both registration & dij: 1064
Total baseline EV sessions/day (sum D): 320332.0
Built distance matrix d of size (1064, 5594)


In [41]:
# ==========================================================
# 2. Global parameters from your proposal
# ==========================================================

# Capacity per charger (sessions/day)
μ_per_charger = 3.65
μ = fill(μ_per_charger, nJ)          # µ_j is homogeneous

# Fixed & variable cost parameters
F = fill(25_000.0, nJ)               # F_j = $25,000
v = fill(15_000.0, nJ)               # v_j = $15,000 per charger

# Detour + environmental cost coefficients
β = 0.7                              # $/km
α = 0.013                            # $/km
cost_per_km = β + α                  # = 0.713 $/km

# Budget and big-M
B = 1_000_000_000.0                  # $1B
M = 50.0                             # max chargers per site (tunable upper bound)

max_chargers_per_station = 10

10

In [42]:
using CSV
using DataFrames
using JuMP
using Gurobi
using Statistics
using Dates   # for timestamp
using FileIO   # for mkpath

function solve_adaptive_model(R_max_km::Float64;
                              write_outputs::Bool = false,
                              tag::String = "")

    λ_unmet = 15000.0   # SAME AS BASELINE

    # ---------- Feasible (i,j) edges within R_max ----------
    edges = Tuple{Int,Int}[]
    for i in I, j in J
        if d[i,j] <= R_max_km
            push!(edges, (i,j))
        end
    end

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    println("R_max = $(R_max_km) km → feasible (i,j) pairs: ", length(edges))

    # ---------- Build JuMP model ----------
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 600.0)
    set_optimizer_attribute(model, "MIPGap", 0.01)

    # First-stage decisions
    @variable(model, y[j in J], Bin)
    @variable(model, z[j in J] >= 0, Int)

    # Second-stage scenario-dependent
    @variable(model, x[(i,j) in edges, s in 1:S] >= 0)
    @variable(model, 0 .<= u[i in I, s in 1:S] .<= 1)

    # ---------- Constraints ----------
    # Demand balance per scenario
    @constraint(model, demand_balance[i in I, s in 1:S],
        sum(x[(i,j), s] for j in N_i[i]) + u[i,s] == 1
    )

    # Capacity: same as baseline but per scenario
    @constraint(model, capacity[j in J, s in 1:S],
        sum(demand[i,s] * x[(i,j), s] for i in N_j[j]) <= μ[j] * z[j]
    )

    # Assignment link
    @constraint(model, assignment_link[(i,j) in edges, s in 1:S],
        x[(i,j), s] <= y[j]
    )

    # Charger/opening links
    @constraint(model, charger_link[j in J], z[j] <= M * y[j])
    @constraint(model, charger_cap[j in J], z[j] <= max_chargers_per_station)

    # Budget (same)
    @constraint(model, budget,
        sum(F[j] * y[j] + v[j] * z[j] for j in J) <= B
    )

    # ---------- Objective: expected cost ----------
    @objective(model, Min,
        sum(F[j] * y[j] + v[j] * z[j] for j in J)
        +
        sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * x[(i,j),s]
            for (i,j) in edges, s in 1:S)
        +
        sum(prob[s] * λ_unmet * demand[i,s] * u[i,s]
            for i in I, s in 1:S)
    )

    optimize!(model)

    term_status = termination_status(model)
    obj_val = objective_value(model)

    println("Termination status: ", term_status)
    println("Objective value:    ", obj_val)

    # =====================================================
    # EXTRACT RESULTS (same as baseline)
    # =====================================================

    open_sites_idx = [j for j in J if value(y[j]) > 0.5]
    total_chargers = sum(value(z[j]) for j in J)

    # Expected costs
    total_fixed = sum(F[j] * value(y[j]) for j in J)
    total_var   = sum(v[j] * value(z[j]) for j in J)

    total_detour_cost =
        sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * value(x[(i,j),s])
            for (i,j) in edges, s in 1:S)

    total_unmet_penalty =
        sum(prob[s] * λ_unmet * demand[i,s] * value(u[i,s])
            for i in I, s in 1:S)

    # Total expected demand served
    total_served =
        sum(prob[s] * demand[i,s] * (1 - value(u[i,s])) for i in I, s in 1:S)

    total_unmet =
        sum(prob[s] * demand[i,s] * value(u[i,s]) for i in I, s in 1:S)

    served_fraction = total_served / (total_served + total_unmet)

    println("---- Adaptive summary for R_max = $(R_max_km) km ----")
    println("Open stations:        ", length(open_sites_idx))
    println("Total chargers:       ", total_chargers)
    println("Total fixed cost:     \$", round(total_fixed; digits=2))
    println("Total charger cost:   \$", round(total_var; digits=2))
    println("Detour+env cost:      \$", round(total_detour_cost; digits=2))
    println("Unmet demand penalty: \$", round(total_unmet_penalty; digits=2))
    println("Expected objective:   \$", round(obj_val; digits=2))
    println("Expected served:      ", round(total_served; digits=2))
    println("Expected unmet:       ", round(total_unmet; digits=2))
    println("Served fraction:      ", round(served_fraction; digits=3))
    println("------------------------------------------------------")

    # =====================================================
    # Build decisions DataFrame (same as baseline)
    # =====================================================
    decisions = DataFrame(
        model_type = String[],
        R_max      = Float64[],
        var_type   = String[],
        ZIP        = Union{Missing,Int}[],
        station_id = Union{Missing,Int}[],
        value      = Float64[],
        scenario   = Union{Missing,Int}[]
    )

    # y_j (binary)
    for j in J
        push!(decisions, ("adaptive", R_max_km, "y",
            missing, station_ids[j], value(y[j]), missing))
    end

    # z_j (chargers)
    for j in J
        push!(decisions, ("adaptive", R_max_km, "z",
            missing, station_ids[j], value(z[j]), missing))
    end

    # u_i,s (unmet)
    for i in I, s in 1:S
        push!(decisions, ("adaptive", R_max_km, "u",
            zip_codes[i], missing, value(u[i,s]), s))
    end

    # x_i,j,s (flows)
    tol = 1e-6
    for (i,j) in edges, s in 1:S
        xv = value(x[(i,j), s])
        if xv > tol
            push!(decisions, ("adaptive", R_max_km, "x",
                zip_codes[i], station_ids[j], xv, s))
        end
    end

    # =====================================================
    # chosen_stations DataFrame (same as baseline)
    # =====================================================
    info_map = Dict(row.ID => row for row in eachrow(stations_small))

    chosen_stations = DataFrame(
        station_id   = Int[],
        status_code  = String[],
        total_evse   = Float64[],
        chargers_opt = Float64[]
    )

    for j in open_sites_idx
        sid = station_ids[j]
        row = info_map[sid]
        push!(chosen_stations, (
            sid,
            row."Status Code",
            row.total_evse,
            value(z[j])
        ))
    end

    # =====================================================
    # assignments DataFrame
    # =====================================================
    assignments = DataFrame(
        ZIP         = Int[],
        station_id  = Int[],
        scenario    = Int[],
        frac_demand = Float64[]
    )

    for (i,j) in edges, s in 1:S
        xv = value(x[(i,j), s])
        if xv > 1e-4
            push!(assignments, (
                zip_codes[i],
                station_ids[j],
                s,
                xv
            ))
        end
    end

    # =====================================================
    # Write outputs to directory
    # =====================================================
    if write_outputs
        outdir = "../data/adaptive"
        mkpath(outdir)

        tag_clean = isempty(tag) ? "R$(Int(R_max_km))" : tag

        CSV.write("$outdir/chosen_stations_adaptive_$(tag_clean).csv", chosen_stations)
        CSV.write("$outdir/assignments_adaptive_$(tag_clean).csv", assignments)
        CSV.write("$outdir/decisions_adaptive_$(tag_clean).csv", decisions)

        summary_df = DataFrame(
            R_max = R_max_km,
            open_sites = length(open_sites_idx),
            total_chargers = total_chargers,
            total_fixed = total_fixed,
            total_var = total_var,
            total_detour_cost = total_detour_cost,
            total_unmet_penalty = total_unmet_penalty,
            expected_served = total_served,
            expected_unmet = total_unmet,
            served_fraction = served_fraction,
            objective = obj_val,
            term_status = string(term_status)
        )
        CSV.write("$outdir/summary_adaptive_$(tag_clean).csv", summary_df)
    end

    return (
        model = model,
        decisions = decisions,
        chosen_stations = chosen_stations,
        assignments = assignments
    )
end


solve_adaptive_model (generic function with 1 method)

In [43]:
Rmax_values = [15.0]
results = Dict{Float64,Any}()

for R in Rmax_values
    println("\n=== Solving baseline model for R_max = $R km ===")
    res = solve_baseline_model(R; write_outputs = true)
    results[R] = res
end


=== Solving baseline model for R_max = 15.0 km ===
R_max = 15.0 km → feasible (i,j) pairs: 186729
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  600
MIPGap  0.01

Optimize a model with 204576 rows, 198981 columns and 781544 nonzeros
Model fingerprint: 0x72386258
Variable types: 187793 continuous, 11188 integer (5594 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+04]
  Objective range  [2e-02, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
Found heuristic solution: objective 4.804980e+09
Presolve removed 6390 rows and 885 columns (presolve

# Adaptive (2025)

In [44]:
println("\n=== START ADAPTIVE MODEL 1204 ===")

adaptive = CSV.read("../data/adaptive_1204.csv", DataFrame)
rename!(adaptive, Dict("ZIP Code" => "ZIP"))

# Ensure ZIP is Int
if eltype(adaptive.ZIP) <: AbstractFloat
    adaptive.ZIP = round.(Int, adaptive.ZIP)
elseif eltype(adaptive.ZIP) <: AbstractString
    adaptive.ZIP = parse.(Int, strip.(adaptive.ZIP))
end

# Keep only ZIPs used in the model
adaptive = adaptive[in.(adaptive.ZIP, Ref(Set(zip_codes))), :]

println("Adaptive rows after ZIP filter = ", nrow(adaptive))

scenario_ids = sort(unique(adaptive.scenario))
S = length(scenario_ids)
println("Scenarios found: ", scenario_ids)

scen_to_idx = Dict(sid => s for (s, sid) in enumerate(scenario_ids))
zip_to_i    = Dict(z => i for (i, z) in enumerate(zip_codes))

# demand[i,s] = 0.25 * predicted EV stock in 2025
sessions_per_EV_per_day = 0.25
demand = zeros(Float64, nI, S)
prob   = zeros(Float64, S)

for row in eachrow(adaptive)
    i = zip_to_i[row.ZIP]
    s = scen_to_idx[row.scenario]
    ev = row.demand_2025             # this is EV stock, same meaning as baseline N
    demand[i, s] = sessions_per_EV_per_day * ev
end

# Scenario weights
for sid in scenario_ids
    s = scen_to_idx[sid]
    w = adaptive.scenario_weight[adaptive.scenario .== sid]
    prob[s] = mean(w)
end
prob ./= sum(prob)

println("Scenario probabilities: ", prob)



=== START ADAPTIVE MODEL 1204 ===
Adaptive rows after ZIP filter = 3177
Scenarios found: [1, 2, 3]
Scenario probabilities: [0.30605800000000016, 0.31104400000000043, 0.38289799999999946]


In [49]:
function solve_adaptive_model(R_max_km::Float64;
                              write_outputs::Bool = false,
                              tag::String = "")

    λ_unmet = 15000.0   # SAME as baseline

    # ---------- Feasible (i,j) edges within R_max ----------
    edges = Tuple{Int,Int}[]
    for i in I, j in J
        if d[i,j] <= R_max_km
            push!(edges, (i,j))
        end
    end

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    println("R_max = $(R_max_km) km → feasible (i,j) pairs: ", length(edges))

    # ---------- Build JuMP model ----------
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 600.0)
    set_optimizer_attribute(model, "MIPGap", 0.01)

    # First-stage decisions (scenario-independent)
    @variable(model, y[j in J], Bin)         # open station j
    @variable(model, z[j in J] >= 0, Int)    # chargers at j

    # Second-stage (scenario-dependent)
    @variable(model, x[edges, s in 1:S] >= 0)              # fraction of demand i served by j in scenario s
    @variable(model, 0 .<= u[i in I, s in 1:S] .<= 1)      # unmet fraction at i in scenario s

    # ---------- Constraints ----------
    # Demand balance: served + unmet = 1 for each i,s
    @constraint(model, demand_balance[i in I, s in 1:S],
        sum(x[(i,j), s] for j in N_i[i]) + u[i,s] == 1
    )

    # Capacity: for each j,s
    @constraint(model, capacity[j in J, s in 1:S],
        sum(demand[i,s] * x[(i,j), s] for i in N_j[j]) <= μ[j] * z[j]
    )

    # Assignment link: x <= y
    @constraint(model, assignment_link[(i,j) in edges, s in 1:S],
        x[(i,j), s] <= y[j]
    )

    # Charger–opening link + hard cap
    @constraint(model, charger_link[j in J],
        z[j] <= M * y[j]
    )
    @constraint(model, charger_cap[j in J],
        z[j] <= max_chargers_per_station
    )

    # Budget (same as baseline)
    @constraint(model, budget,
        sum(F[j] * y[j] + v[j] * z[j] for j in J) <= B
    )

    # ---------- Objective: expected cost ----------
    @objective(model, Min,
        # infra cost (scenario-independent)
        sum(F[j] * y[j] + v[j] * z[j] for j in J)
        +
        # expected detour+env cost
        sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * x[(i,j), s]
            for (i,j) in edges, s in 1:S)
        +
        # expected unmet demand penalty
        sum(prob[s] * λ_unmet * demand[i,s] * u[i,s] for i in I, s in 1:S)
    )

    # ---------- Solve ----------
    optimize!(model)

    term_status = termination_status(model)
    println("Termination status: ", term_status)
    obj_val = objective_value(model)
    println("Objective value:    ", obj_val)

    # =====================================================
    #  Extract summary stats (this was missing before!)
    # =====================================================

    # Stations opened and chargers
    open_sites_idx = [j for j in J if value(y[j]) > 0.5]
    num_open = length(open_sites_idx)
    total_chargers = sum(value(z[j]) for j in J)

    # Costs
    total_fixed = sum(F[j] * value(y[j]) for j in J)
    total_var   = sum(v[j] * value(z[j]) for j in J)

    total_detour_cost =
        sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * value(x[(i,j), s])
            for (i,j) in edges, s in 1:S)

    total_unmet_penalty =
        sum(prob[s] * λ_unmet * demand[i,s] * value(u[i,s])
            for i in I, s in 1:S)

    # Expected served / unmet demand
    total_served =
        sum(prob[s] * demand[i,s] * (1.0 - value(u[i,s])) for i in I, s in 1:S)

    total_unmet =
        sum(prob[s] * demand[i,s] * value(u[i,s]) for i in I, s in 1:S)

    served_fraction = total_served / max(total_served + total_unmet, 1e-9)

    println("---- Adaptive summary for R_max = $(R_max_km) km ----")
    println("Open stations:        ", num_open)
    println("Total chargers:       ", total_chargers)
    println("Total fixed cost:     \$", round(total_fixed; digits=2))
    println("Total charger cost:   \$", round(total_var; digits=2))
    println("Detour+env cost:      \$", round(total_detour_cost; digits=2))
    println("Unmet demand penalty: \$", round(total_unmet_penalty; digits=2))
    println("Expected objective:   \$", round(obj_val; digits=2))
    println("Expected served:      ", round(total_served; digits=2))
    println("Expected unmet:       ", round(total_unmet; digits=2))
    println("Served fraction:      ", round(served_fraction; digits=3))
    println("------------------------------------------------------")

    # =====================================================
    #  Build decisions DataFrame (like baseline)
    # =====================================================
    decisions = DataFrame(
        model_type = String[],
        R_max      = Float64[],
        var_type   = String[],
        ZIP        = Union{Missing,Int}[],
        station_id = Union{Missing,Int}[],
        value      = Float64[],
        scenario   = Union{Missing,Int}[]
    )

    # y_j
    for j in J
        push!(decisions, (
            "adaptive",
            R_max_km,
            "y",
            missing,
            station_ids[j],
            value(y[j]),
            missing
        ))
    end

    # z_j
    for j in J
        push!(decisions, (
            "adaptive",
            R_max_km,
            "z",
            missing,
            station_ids[j],
            value(z[j]),
            missing
        ))
    end

    # u_i,s
    for i in I, s in 1:S
        push!(decisions, (
            "adaptive",
            R_max_km,
            "u",
            zip_codes[i],
            missing,
            value(u[i,s]),
            s
        ))
    end

    # x_i,j,s  (only store non-tiny)
    tol = 1e-6
    for (i,j) in edges, s in 1:S
        xv = value(x[(i,j), s])
        if xv > tol
            push!(decisions, (
                "adaptive",
                R_max_km,
                "x",
                zip_codes[i],
                station_ids[j],
                xv,
                s
            ))
        end
    end

    # =====================================================
    # chosen_stations & assignments (like baseline)
    # =====================================================
    info_map = Dict(row.ID => row for row in eachrow(stations_small))

    chosen_stations = DataFrame(
        station_id   = Int[],
        status_code  = String[],
        total_evse   = Float64[],
        chargers_opt = Float64[]
    )

    for j in open_sites_idx
        sid = station_ids[j]
        row = info_map[sid]
        push!(chosen_stations, (
            sid,
            row."Status Code",
            row.total_evse,
            value(z[j])
        ))
    end

    assignments = DataFrame(
        ZIP         = Int[],
        station_id  = Int[],
        scenario    = Int[],
        frac_demand = Float64[]
    )

    for (i,j) in edges, s in 1:S
        xv = value(x[(i,j), s])
        if xv > 1e-4
            push!(assignments, (
                zip_codes[i],
                station_ids[j],
                s,
                xv
            ))
        end
    end

    # =====================================================
    # Optional: write outputs
    # =====================================================
    if write_outputs
        outdir = "../data/adaptive"
        mkpath(outdir)

        suffix = isempty(tag) ? "R$(Int(R_max_km))" : tag

        CSV.write("$outdir/chosen_stations_$(suffix).csv", chosen_stations)
        CSV.write("$outdir/assignments_$(suffix).csv", assignments)
        CSV.write("$outdir/decisions_$(suffix).csv", decisions)

        summary_df = DataFrame(
            R_max = R_max_km,
            open_sites = num_open,
            total_chargers = total_chargers,
            total_fixed = total_fixed,
            total_var = total_var,
            total_detour_cost = total_detour_cost,
            total_unmet_penalty = total_unmet_penalty,
            expected_served = total_served,
            expected_unmet = total_unmet,
            served_fraction = served_fraction,
            objective = obj_val,
            term_status = string(term_status)
        )
        CSV.write("$outdir/summary_$(suffix).csv", summary_df)
    end

    return (
        model = model,
        decisions = decisions,
        chosen_stations = chosen_stations,
        assignments = assignments,
        summary = (
            R_max = R_max_km,
            open_sites = num_open,
            total_chargers = total_chargers,
            total_fixed = total_fixed,
            total_var = total_var,
            total_detour_cost = total_detour_cost,
            total_unmet_penalty = total_unmet_penalty,
            expected_served = total_served,
            expected_unmet = total_unmet,
            served_fraction = served_fraction,
            objective = obj_val,
            term_status = term_status
        )
    )
end


solve_adaptive_model (generic function with 1 method)

In [50]:
Rmax_values = [15.0]
adaptive_results = Dict{Float64,Any}()

for R in Rmax_values
    println("\n=== Solving ADAPTIVE model for R_max = $R km ===")
    adaptive_results[R] = solve_adaptive_model(R; write_outputs=false)
end



=== Solving ADAPTIVE model for R_max = 15.0 km ===
R_max = 15.0 km → feasible (i,j) pairs: 186729
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  600
MIPGap  0.01

Optimize a model with 591350 rows, 574567 columns and 2284777 nonzeros
Model fingerprint: 0xd15e6d1b
Variable types: 563379 continuous, 11188 integer (5594 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [3e-02, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
Found heuristic solution: objective 6.965913e+09
Presolve removed 12760 rows and 6818 columns (preso

# Deterministic

In [71]:
function solve_deterministic_point(R_max_km::Float64)

    s0 = 2   # Scenario 2 is the point-forecast

    println("\n=== Solving DETERMINISTIC (Scenario 2) for R_max = $R_max_km ===")

    # ---- Feasible edges ----
    edges = [(i,j) for i in I, j in J if d[i,j] <= R_max_km]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    λ_unmet = 15000.0

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 600.0)
    set_optimizer_attribute(model, "MIPGap", 0.01)

    # First-stage decisions
    @variable(model, y[j in J], Bin)
    @variable(model, z[j in J] >= 0, Int)

    # Second-stage (single scenario)
    @variable(model, x[(i,j) in edges] >= 0)
    @variable(model, 0 .<= u[i in I] .<= 1)

    # Constraints
    @constraint(model, [i in I],
        sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
    )

    @constraint(model, [j in J],
        sum(demand[i,s0] * x[(i,j)] for i in N_j[j]) <= μ[j] * z[j]
    )

    @constraint(model, [i in I, j in N_i[i]],
        x[(i,j)] <= y[j]
    )

    @constraint(model, [j in J], z[j] <= M * y[j])
    @constraint(model, [j in J], z[j] <= max_chargers_per_station)

    @constraint(model,
        sum(F[j]*y[j] + v[j]*z[j] for j in J) <= B
    )

    # Objective
    @objective(model, Min,
        sum(F[j]*y[j] + v[j]*z[j] for j in J)
        +
        sum(cost_per_km * d[i,j] * demand[i,s0] * x[(i,j)] for (i,j) in edges)
        +
        sum(λ_unmet * demand[i,s0] * u[i] for i in I)
    )

    optimize!(model)

    term_status = termination_status(model)
    obj_val = objective_value(model)

    # Extract first-stage decisions
    y_opt = Dict(j => value(y[j]) for j in J)
    z_opt = Dict(j => value(z[j]) for j in J)

    println("Deterministic objective = ", obj_val)

    return (model = model, y_opt = y_opt, z_opt = z_opt)
end


solve_deterministic_point (generic function with 1 method)

In [90]:
function evaluate_deterministic_first_stage(y_opt, z_opt, R_max_km)

    println("\n=== Evaluating deterministic decisions under all scenarios ===")

    S_costs = zeros(S)
    served = zeros(S)
    unmet = zeros(S)

    # Build edges again
    edges = [(i,j) for i in I, j in J if d[i,j] <= R_max_km]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    λ_unmet = 15000.0

    # Evaluate deterministic decisions under each scenario
    for s in 1:S

        model = Model(Gurobi.Optimizer)

        @variable(model, x[(i,j) in edges] >= 0)
        @variable(model, 0 .<= u[i in I] .<= 1)

        @constraint(model, [i in I],
            sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
        )

        @constraint(model, [j in J],
            sum(demand[i,s] * x[(i,j)] for i in N_j[j]) <= μ[j] * z_opt[j]
        )

        @constraint(model, [i in I, j in N_i[i]],
            x[(i,j)] <= y_opt[j]
        )

        @objective(model, Min,
            sum(F[j]*y_opt[j] + v[j]*z_opt[j] for j in J)
            +
            sum(cost_per_km * d[i,j] * demand[i,s] * x[(i,j)] for (i,j) in edges)
            +
            sum(λ_unmet * demand[i,s] * u[i] for i in I)
        )

        optimize!(model)

        S_costs[s] = objective_value(model)
        served[s] = sum(demand[i,s]*(1 - value(u[i])) for i in I)
        unmet[s]  = sum(demand[i,s]*value(u[i]) for i in I)

        println(" Scenario $s cost = ", S_costs[s])
    end

    expected_cost = sum(prob[s] * S_costs[s] for s in 1:S)
    println("Expected deterministic cost = ", expected_cost)

    # --------------------------------------------
    # Build consistent summary object
    # --------------------------------------------
    total_fixed = sum(F[j] * y_opt[j] for j in J)
    total_var   = sum(v[j] * z_opt[j] for j in J)

    # Expected detour + unmet costs
    total_detour_cost =
        sum(prob[s] * S_costs[s] for s in 1:S) - (total_fixed + total_var)

    total_unmet_penalty =
        sum(prob[s] * unmet[s] * λ_unmet for s in 1:S)

    total_served = sum(prob[s] * served[s] for s in 1:S)
    total_unmet  = sum(prob[s] * unmet[s] for s in 1:S)

    served_fraction = total_served / (total_served + total_unmet + 1e-9)

    summary = (
        R_max = R_max_km,
        total_fixed = total_fixed,
        total_var = total_var,
        total_detour_cost = total_detour_cost,
        total_unmet_penalty = total_unmet_penalty,
        expected_served = total_served,
        expected_unmet = total_unmet,
        served_fraction = served_fraction,
        objective = expected_cost,
        term_status = "EVALUATED"
    )

    return summary
end


evaluate_deterministic_first_stage (generic function with 1 method)

In [91]:
function evaluate_deterministic_first_stage(y_opt, z_opt, R_max_km)

    println("\n=== Evaluating deterministic decisions under all scenarios ===")

    S_costs = zeros(S)
    served = zeros(S)
    unmet = zeros(S)

    # Build edges again
    edges = [(i,j) for i in I, j in J if d[i,j] <= R_max_km]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    λ_unmet = 15000.0

    # Evaluate deterministic decisions under each scenario
    for s in 1:S

        model = Model(Gurobi.Optimizer)

        @variable(model, x[(i,j) in edges] >= 0)
        @variable(model, 0 .<= u[i in I] .<= 1)

        @constraint(model, [i in I],
            sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
        )

        @constraint(model, [j in J],
            sum(demand[i,s] * x[(i,j)] for i in N_j[j]) <= μ[j] * z_opt[j]
        )

        @constraint(model, [i in I, j in N_i[i]],
            x[(i,j)] <= y_opt[j]
        )

        @objective(model, Min,
            sum(F[j]*y_opt[j] + v[j]*z_opt[j] for j in J)
            +
            sum(cost_per_km * d[i,j] * demand[i,s] * x[(i,j)] for (i,j) in edges)
            +
            sum(λ_unmet * demand[i,s] * u[i] for i in I)
        )

        optimize!(model)

        S_costs[s] = objective_value(model)
        served[s] = sum(demand[i,s]*(1 - value(u[i])) for i in I)
        unmet[s]  = sum(demand[i,s]*value(u[i]) for i in I)

        println(" Scenario $s cost = ", S_costs[s])
    end

    expected_cost = sum(prob[s] * S_costs[s] for s in 1:S)
    println("Expected deterministic cost = ", expected_cost)

    # --------------------------------------------
    # Build consistent summary object
    # --------------------------------------------
    total_fixed = sum(F[j] * y_opt[j] for j in J)
    total_var   = sum(v[j] * z_opt[j] for j in J)

    # Expected unmet penalty
    total_unmet_penalty =
        sum(prob[s] * unmet[s] * λ_unmet for s in 1:S)

    # Expected served / unmet demand
    total_served = sum(prob[s] * served[s] for s in 1:S)
    total_unmet  = sum(prob[s] * unmet[s] for s in 1:S)

    served_fraction = total_served / (total_served + total_unmet + 1e-9)

    # Expected detour cost (only detour, no unmet)
    total_detour_cost =
        expected_cost - total_fixed - total_var - total_unmet_penalty

    expected_cost = sum(prob[s] * S_costs[s] for s in 1:S)
    println("Expected deterministic cost = ", expected_cost)

    summary = (
        R_max = R_max_km,
        total_fixed = total_fixed,
        total_var = total_var,
        total_detour_cost = total_detour_cost,
        total_unmet_penalty = total_unmet_penalty,
        expected_served = total_served,
        expected_unmet = total_unmet,
        served_fraction = served_fraction,
        objective = expected_cost,
        term_status = "EVALUATED"
    )

    return (
        S_costs = S_costs,
        served_per_scenario = served,
        unmet_per_scenario = unmet,
        expected_cost = expected_cost,
        summary = summary
    )
    end


evaluate_deterministic_first_stage (generic function with 1 method)

In [73]:
det_first = solve_deterministic_point(15.0)
det_eval = evaluate_deterministic_first_stage(det_first.y_opt, det_first.z_opt, 15.0)



=== Solving DETERMINISTIC (Scenario 2) for R_max = 15.0 ===
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  600
MIPGap  0.01

Optimize a model with 204576 rows, 198981 columns and 780239 nonzeros
Model fingerprint: 0xf2963cab
Variable types: 187793 continuous, 11188 integer (5594 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+04]
  Objective range  [1e-01, 8e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
Found heuristic solution: objective 7.084966e+09
Presolve removed 7762 rows and 2229 columns (presolve time = 7s)...
Presolve removed 7762 

(R_max = 15.0, total_fixed = 1.2975e8, total_var = 7.737e8, total_detour_cost = 4.192993370819106e9, total_unmet_penalty = 4.192640105502821e9, expected_served = 184884.88129849388, expected_unmet = 279509.3403668547, served_fraction = 0.39812054645185724, objective = 5.096443370819106e9, term_status = "EVALUATED")

# Wait and See

In [81]:
function solve_wait_and_see(R_max_km::Float64)

    println("\n=== Solving WAIT-AND-SEE for R_max = $R_max_km ===")

    # ------------------------------------------
    # Build feasible edges
    # ------------------------------------------
    edges = [(i,j) for i in I, j in J if d[i,j] <= R_max_km]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    λ_unmet = 15000.0

    # ------------------------------------------
    # Storage across scenarios
    # ------------------------------------------
    cost_scenario = zeros(S)
    served_s      = zeros(S)
    unmet_s       = zeros(S)

    y_WS = Dict{Int, Vector{Float64}}()
    z_WS = Dict{Int, Vector{Float64}}()

    assignments = DataFrame(
        ZIP = Int[],
        station_id = Int[],
        scenario = Int[],
        frac_demand = Float64[]
    )

    # ------------------------------------------
    # Solve S independent models
    # ------------------------------------------
    for s in 1:S
        println("  -- Scenario $s --")

        model = Model(Gurobi.Optimizer)
        set_optimizer_attribute(model, "TimeLimit", 600.0)
        set_optimizer_attribute(model, "MIPGap", 0.01)

        @variable(model, y[j in J], Bin)
        @variable(model, z[j in J] >= 0, Int)
        @variable(model, x[(i,j) in edges] >= 0)
        @variable(model, 0 .<= u[i in I] .<= 1)

        @constraint(model, [i in I],
            sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
        )

        @constraint(model, [j in J],
            sum(demand[i,s] * x[(i,j)] for i in N_j[j]) <= μ[j] * z[j]
        )

        @constraint(model, [i in I, j in N_i[i]],
            x[(i,j)] <= y[j]
        )

        @constraint(model, [j in J], z[j] <= M*y[j])
        @constraint(model, [j in J], z[j] <= max_chargers_per_station)

        @constraint(model,
            sum(F[j]*y[j] + v[j]*z[j] for j in J) <= B
        )

        @objective(model, Min,
            sum(F[j]*y[j] + v[j]*z[j] for j in J) +
            sum(cost_per_km * d[i,j] * demand[i,s] * x[(i,j)] for (i,j) in edges) +
            sum(λ_unmet * demand[i,s] * u[i] for i in I)
        )

        optimize!(model)

        # --- store results
        cost_scenario[s] = objective_value(model)
        served_s[s] = sum(demand[i,s] * (1 - value(u[i])) for i in I)
        unmet_s[s]  = sum(demand[i,s] * value(u[i]) for i in I)

        y_WS[s] = [value(y[j]) for j in J]
        z_WS[s] = [value(z[j]) for j in J]

        # assignments
        for (i,j) in edges
            xv = value(x[(i,j)])
            if xv > 1e-4
                push!(assignments, (
                    zip_codes[i],
                    station_ids[j],
                    s,
                    xv
                ))
            end
        end

        println("     WS scenario cost = ", cost_scenario[s])
    end

    # ------------------------------------------
    # Expected cost
    # ------------------------------------------
    expected_WS = sum(prob[s] * cost_scenario[s] for s in 1:S)
    println("\nExpected Wait-and-See cost = ", expected_WS)

    # ------------------------------------------
    # Build station list (scenario-wise)
    # ------------------------------------------
    chosen_stations = DataFrame(
        scenario = Int[],
        station_id = Int[],
        chargers_opt = Float64[]
    )

    for s in 1:S
        for j in J
            if y_WS[s][j] > 0.5
                push!(chosen_stations, (
                    s,
                    station_ids[j],
                    z_WS[s][j]
                ))
            end
        end
    end

    # ------------------------------------------
    # Compute exp fixed, var, detour, unmet
    # ------------------------------------------
    total_fixed =
        sum(prob[s] * sum(F[j] * y_WS[s][j] for j in J) for s in 1:S)

    total_var =
        sum(prob[s] * sum(v[j] * z_WS[s][j] for j in J) for s in 1:S)

    # unmet penalty
    total_unmet_penalty =
        sum(prob[s] * unmet_s[s] * λ_unmet for s in 1:S)

    served_exp = sum(prob[s] * served_s[s] for s in 1:S)
    unmet_exp  = sum(prob[s] * unmet_s[s] for s in 1:S)

    served_fraction = served_exp / (served_exp + unmet_exp + 1e-9)

    total_detour_cost =
        expected_WS - total_fixed - total_var - total_unmet_penalty

    # ------------------------------------------
    # Summary
    # ------------------------------------------
    summary = (
        R_max = R_max_km,
        total_fixed = total_fixed,
        total_var = total_var,
        total_detour_cost = total_detour_cost,
        total_unmet_penalty = total_unmet_penalty,
        expected_served = served_exp,
        expected_unmet = unmet_exp,
        served_fraction = served_fraction,
        objective = expected_WS,
        term_status = "WAIT_AND_SEE"
    )

    # ------------------------------------------
    # Decisions DF
    # ------------------------------------------
    decisions = DataFrame(
        model_type = String[],
        R_max = Float64[],
        var_type = String[],
        ZIP = Union{Missing,Int}[],
        station_id = Union{Missing,Int}[],
        value = Float64[],
        scenario = Union{Missing,Int}[]
    )

    for s in 1:S, j in J
        push!(decisions, ("wait_and_see", R_max_km, "y",
            missing, station_ids[j], y_WS[s][j], s))
        push!(decisions, ("wait_and_see", R_max_km, "z",
            missing, station_ids[j], z_WS[s][j], s))
    end

    return (
        decisions = decisions,
        chosen_stations = chosen_stations,
        assignments = assignments,
        summary = summary
    )
end


solve_wait_and_see (generic function with 1 method)

In [82]:
ws = solve_wait_and_see(15.0)



=== Solving WAIT-AND-SEE for R_max = 15.0 ===
  -- Scenario 1 --
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  600
MIPGap  0.01

Optimize a model with 204576 rows, 198981 columns and 780239 nonzeros
Model fingerprint: 0x7ad28c22
Variable types: 187793 continuous, 11188 integer (5594 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [9e-02, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
Found heuristic solution: objective 5.494894e+09
Presolve removed 7396 rows and 1883 columns
Presolve time: 3.69s
Presolved: 197180 ro

(decisions = 33564×7 DataFrame
   Row │ model_type    R_max    var_type  ZIP      station_id  value    scenar ⋯
       │ String        Float64  String    Int64?   Int64?      Float64  Int64? ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │ wait_and_see     15.0  y         missing       35272      1.0         ⋯
     2 │ wait_and_see     15.0  z         missing       35272     10.0
     3 │ wait_and_see     15.0  y         missing       35562      1.0
     4 │ wait_and_see     15.0  z         missing       35562     10.0
     5 │ wait_and_see     15.0  y         missing       35563      1.0         ⋯
     6 │ wait_and_see     15.0  z         missing       35563     10.0
     7 │ wait_and_see     15.0  y         missing       36532      1.0
     8 │ wait_and_see     15.0  z         missing       36532     10.0
   ⋮   │      ⋮           ⋮        ⋮         ⋮         ⋮          ⋮        ⋮   ⋱
 33558 │ wait_and_see     15.0  z         missing      44

In [83]:
using DataFrames

function build_summary_table(det, adapt, ws)

    return DataFrame(
        Model = ["Deterministic", "Adaptive", "Wait-and-See"],
        Objective_Cost = [det.objective, adapt.objective, ws.objective],
        Fixed_Cost     = [det.total_fixed, adapt.total_fixed, ws.total_fixed],
        Variable_Cost  = [det.total_var, adapt.total_var, ws.total_var],
        Detour_Cost    = [det.total_detour_cost, adapt.total_detour_cost, ws.total_detour_cost],
        Unmet_Penalty  = [det.total_unmet_penalty, adapt.total_unmet_penalty, ws.total_unmet_penalty],
        Served         = [det.expected_served, adapt.expected_served, ws.expected_served],
        Unmet          = [det.expected_unmet, adapt.expected_unmet, ws.expected_unmet],
        Served_Frac    = [det.served_fraction, adapt.served_fraction, ws.served_fraction]
    )
end


build_summary_table (generic function with 2 methods)

In [76]:
function build_ev_summary(det, ada, ws)

    return DataFrame(
        Model = [
            "Deterministic (EV)",
            "Adaptive (Stochastic)",
            "Wait-and-See"
        ],

        R_max = [
            det.R_max,
            ada.R_max,
            ws.R_max
        ],

        Expected_Cost = [
            det.expected_cost,
            ada.objective,          # adaptive has .objective instead
            ws.expected_cost
        ],

        Total_Fixed_Cost = [
            det.total_fixed,
            ada.total_fixed,
            ws.total_fixed
        ],

        Total_Variable_Cost = [
            det.total_var,
            ada.total_var,
            ws.total_var
        ],

        Unmet_Penalty = [
            det.total_unmet_penalty,
            ada.total_unmet_penalty,
            ws.expected_unmet * 15000.0   # WS does not track penalty explicitly
        ],

        Expected_Served = [
            det.expected_served,
            ada.expected_served,
            ws.expected_served
        ],

        Expected_Unmet = [
            det.expected_unmet,
            ada.expected_unmet,
            ws.expected_unmet
        ],

        Served_Fraction = [
            det.served_fraction,
            ada.served_fraction,
            ws.served_fraction
        ],

        Open_Stations = [
            det.open_sites,
            ada.open_sites,
            missing              # WS opens different stations per scenario
        ],

        Total_Chargers = [
            det.total_chargers,
            ada.total_chargers,
            missing              # WS varies per scenario
        ],

        Termination_Status = [
            string(det.term_status),
            string(ada.term_status),
            string(ws.term_status)
        ]
    )
end


build_ev_summary (generic function with 2 methods)

In [85]:
adaptive_summary = adaptive_results[15.0][:summary]
det_summary      = det_eval                     # already a NamedTuple
ws_summary       = ws.summary                   # ws is a NamedTuple
table = build_summary_table(det_summary, adaptive_summary, ws_summary)
println(table)

3×9 DataFrame
 Row │ Model          Objective_Cost  Fixed_Cost  Variable_Cost  Detour_Cost  Unmet_Penalty  Served     Unmet      Served_Frac
     │ String         Float64         Float64     Float64        Float64      Float64        Float64    Float64    Float64
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Deterministic       5.09644e9   1.2975e8       7.737e8      3.53265e5      4.19264e9  1.84885e5  2.79509e5     0.398121
   2 │ Adaptive            5.09183e9   1.3285e8       7.93695e8    3.69547e5      4.16491e9  1.86733e5  2.77661e5     0.402101
   3 │ Wait-and-See        5.06169e9   1.28917e8      7.67749e8    3.69912e5      4.16465e9  1.86751e5  2.77643e5     0.402139
